In [1]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hl
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict,Set
import uuid
import json
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from gnomad.utils.plotting import *
from typing import Set, Tuple
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
import warnings; 
warnings.simplefilter('ignore')
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")
output_notebook()

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/exomeQC-hail-gnomad/notebooks/hail-20210629-0912-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

In [2]:
mt=hl.read_matrix_table(f'{lustre_dir}/MegaWESSanger_cohorts_sampleQC_filtered.mt')

In [3]:
mt.count()

(28803842, 93307)

In [4]:
mt.summarize()

2021-06-29 08:42:26 Hail: INFO: Coerced sorted dataset


Non-missing,93307 (100.00%)
Missing,0
Min Size,10
Max Size,22
Mean Size,18.72
Sample Values,"['EGAN00001006259', 'EGAN00001006260', 'EGAN00001006261', 'EGAN00001006263', 'EGAN00001006264']"
Non-missing,93307 (100.00%)
Missing,0
Min Size,3
Max Size,16
Mean Size,4.31


Non-missing,28803842 (100.00%)
Missing,0
Contig Counts,"{'chr11': 1632197, 'chr5': 1342615, 'chr22': 696147, 'chr8': 1104562, 'chr19': 1662084, 'chrY': 28704, 'chr1': 2759565, 'chr15': 1077895, 'chr12': 1571754, 'chr18': 563132, 'chr20': 720998, 'chr2': 2099677, 'chr13': 596180, 'chr7': 1431748, 'chr14': 991416, 'chr3': 1653803, 'chrM': 7068, 'chr17': 1555676, 'chr4': 1200680, 'chr6': 1360406, 'chr9': 1159584, 'chrX': 760027, 'chr10': 1147841, 'chr21': 354289, 'chr16': 1325794}"
Non-missing,28803842 (100.00%)
Missing,0
Min Size,2
Max Size,7
Mean Size,2.13
Non-missing,61451819 (100.00%)
Missing,0
Min Size,1


KeyboardInterrupt: 

In [6]:
#truthset_table
truthset_ht=hl.read_table(f'{lustre_dir}/variant_qc/truthset_table.ht')

In [7]:
truthset_ht.count()

30744091

In [8]:
truthset_ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'hapmap': bool 
    'omni': bool 
    'kgp_phase1_hc': bool 
    'mills': bool 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [9]:
trio_dataset=hl.read_matrix_table(f'{lustre_dir}/variant_qc/MegaWES_trios_adj.mt')
trio_dataset.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    'id': str
    'proband': struct {
        s: str, 
        cohort: str
    }
    'father': struct {
        s: str, 
        cohort: str
    }
    'mother': struct {
        s: str, 
        cohort: str
    }
    'is_female': bool
    'fam_id': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: i

In [3]:
trio_stats=hl.read_table(f'{lustre_dir}/variant_qc/MegaWES_stats.ht')
trio_stats.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'n_transmitted_raw': int64 
    'n_untransmitted_raw': int64 
    'n_transmitted_adj': int64 
    'n_untransmitted_adj': int64 
    'n_de_novos_raw': int64 
    'n_de_novos_adj': int64 
    'ac_parents_raw': int64 
    'an_parents_raw': int64 
    'ac_children_raw': int64 
    'an_children_raw': int64 
    'ac_parents_adj': int64 
    'an_parents_adj': int64 
    'ac_children_adj': int64 
    'an_children_adj': int64 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [4]:
trio_stats.summarize()

Non-missing,24859339 (100.00%)
Missing,0
Contig Counts,"{'chr11': 1450261, 'chr5': 1203523, 'chr22': 611414, 'chr8': 982420, 'chr19': 1430073, 'chr1': 2460472, 'chr15': 962622, 'chr12': 1403719, 'chr18': 504803, 'chr20': 635817, 'chr2': 1878570, 'chr13': 532914, 'chr7': 1263584, 'chr14': 885417, 'chr3': 1481294, 'chr17': 1371312, 'chr4': 1071985, 'chr6': 1216311, 'chr9': 1022510, 'chr10': 1022698, 'chr21': 310851, 'chr16': 1156769}"
Non-missing,24859339 (100.00%)
Missing,0
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,49718678 (100.00%)
Missing,0
Min Size,1


In [5]:
trio_stats.show()

,,,,,,,,,,,,,,,
locus,alleles,n_transmitted_raw,n_untransmitted_raw,n_transmitted_adj,n_untransmitted_adj,n_de_novos_raw,n_de_novos_adj,ac_parents_raw,an_parents_raw,ac_children_raw,an_children_raw,ac_parents_adj,an_parents_adj,ac_children_adj,an_children_adj
locus<GRCh38>,array<str>,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
chr1:12938,"[""GCAAA"",""G""]",0,0,0,0,0,0,4,3246,0,1616,0,0,0,0
chr1:13024,"[""G"",""A""]",0,0,0,0,0,0,0,8758,2,4348,0,0,0,0
chr1:13087,"[""A"",""G""]",0,0,0,0,0,0,0,18992,2,9396,0,0,0,0
chr1:13116,"[""T"",""C""]",0,0,0,0,0,0,2,19278,0,9518,0,0,0,0
chr1:13130,"[""C"",""T""]",0,0,0,0,0,0,0,17014,0,8404,0,0,0,0
chr1:13151,"[""G"",""C""]",0,0,0,0,0,0,0,19334,0,9698,0,0,0,0
chr1:13164,"[""G"",""A""]",0,0,0,0,0,0,0,21318,0,10646,0,0,0,0
chr1:13176,"[""G"",""T""]",0,0,0,0,0,0,0,22924,2,11552,0,0,0,0


In [11]:
ht_inbreeding=hl.read_table(f'{lustre_dir}/variant_qc/MegaWES_inbreeding_new.ht')
qc_ac_ht=hl.read_table(f'{lustre_dir}/variant_qc/MegaWES_qc_ac_new.ht')
allele_data_ht=hl.read_table(f'{lustre_dir}/variant_qc/MegaWES_allele_data_new.ht')


In [12]:
ht_inbreeding.count()

28803842

In [13]:
qc_ac_ht.count()

28803842

In [14]:
allele_data_ht.count()

31316024

In [15]:
qc_ac_ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        QD: float64, 
        RAW_MQandDP: array<int32>, 


In [17]:
allele_data_ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'allele_data': struct {
        nonsplit_alleles: array<str>, 
        has_star: bool, 
        variant_type: str, 
        n_alt_alleles: int32, 
        allele_type: str, 
        was_mixed: bool
    } 
    'a_index': int32 
    'was_split': bool 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [18]:
ht_inbreeding.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        QD: float64, 
        RAW_MQandDP: array<int32>, 


In [19]:
#Trio matrixtable
mt_trios=hl.read_matrix_table(f'{lustre_dir}/variant_qc//MegaWES_trio_table.mt')

In [20]:
mt_trios.count()

(31316024, 9742)

In [21]:
mt_trios.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    'id': str
    'proband': struct {
        s: str, 
        cohort: str
    }
    'father': struct {
        s: str, 
        cohort: str
    }
    'mother': struct {
        s: str, 
        cohort: str
    }
    'is_female': bool
    'fam_id': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: i

In [22]:
#Family stats
mt_family_stats=hl.read_matrix_table(f'{lustre_dir}/variant_qc//MegaWES_family_stats.mt')
mt_family_stats.count()


(31316024, 93307)

In [23]:
mt_family_stats.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'cohort': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
     

In [24]:
denovo_table=hl.read_table(f'{lustre_dir}/variant_qc/MegaWES_denovo_table.ht')
denovo_table.count()

301770

In [25]:
denovo_table.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'de_novo_data': array<struct {
        id: str, 
        prior: float64, 
        proband: struct {
            s: str, 
            cohort: str
        }, 
        father: struct {
            s: str, 
            cohort: str
        }, 
        mother: struct {
            s: str, 
            cohort: str
        }, 
        proband_entry: struct {
            AD: array<int32>, 
            DP: int32, 
            GQ: int32, 
            GT: call, 
            MIN_DP: int32, 
            PGT: call, 
            PID: str, 
            PL: array<int32>, 
            PS: int32, 
            RGQ: int32, 
            SB: array<int32>
        }, 
        father_entry: struct {
            AD: array<int32>, 
            DP: int32, 
            GQ: int32, 
            GT: call, 
            MIN_DP: int32, 
           